### Reinforcement Learning: Theory and Algorithms

* 第3章における各定理、補題、系などをまとめます。証明は初見でもこれを参考にすれば導出できるように丁寧に記します。
* 必要に応じて実装を行います。



### Linear Bellman Completeness(LBC) 

* 状態数と行動数が大きい場合(可算あるいは非可算無限)場合にうまく動くアルゴリズムを考えます。
* LBCはある条件であり、これを満たすと多項式的なサンプル複雑度で最適な方策を学習できます。
* 本章では**有限-Horizon**MDPに焦点を当てます。


#### 準備
* 線形関数：$f(s, a) := \theta^{\top} \phi(s, a)$
* 特徴写像：$\phi: \mathcal{S} \times \mathcal{A} \mapsto \mathbb{R}^d$
* 適切な次元の行列 $M$ とベクトル $x$ に対して、$\|x\|_M^2 = x^{\top} M x$ という表記を使用します．


#### Linear Bellman Completeness

特徴 $\phi$ がLinear Bellman Completenessを満たすとは、任意の $\theta \in \mathbb{R}^d$ および $(s, a, h) \in \mathcal{S} \times \mathcal{A} \times[H]$ に対して、次を満たす $w \in \mathbb{R}^d$ が存在することをいいます。

$$
w^{\top} \phi(s, a) = r(s, a) + \mathbb{E}_{s^{\prime} \sim P_h(s, a)} \max_{a^{\prime}} \theta^{\top} \phi(s^{\prime}, a^{\prime}).
$$

$w$ は $\theta$ に依存するため、上記の方程式で $w := \mathcal{T}_h(\theta)$ と表記します。

* 線形関数にベルマン作用素を適用($w^{\top} \phi(s, a) = \mathcal{T}_h(\theta)\phi(s, a)$)しても、適用結果が線形関数で表現できる(右辺)という条件です。つまり、線形関数にベルマン作用素を適用しても非線形関数とかにならないという条件です。
* 線形であることのメリットとして、$Q_h^{\star}(s, a) = (\theta_h^{\star})^{\top} \phi(s, a)$ を満たす $\theta_h^{\star}$ が存在することが挙げられます。つまりLBCを満たせば、最適価値関数は必ず存在するということです。



### LSVIアルゴリズム
最適価値関数$Q_h^{\star}(s, a)$を学習するアルゴリズムを考えましょう．


アルゴリズム 1 最小二乗値反復 (LSVI)








* LSVIがほぼ最適な方策を見つけるためには，**データセット**をいい感じにする必要があります．
* いい感じにするには，各データセット$\mathcal{D}_h$において，$\sum_{s, a \in \mathcal{D}_h} \phi(s, a) \phi(s, a)^{\top}$が**フルランク**になっている必要があります．
  * これにより，最小二乗法を閉じた形で解くことができ，いい感じに汎化することができます．
  * なぜフルランクになると閉じた形になるかというと(**TODO**)
* この条件(フルランク)を達成するデータセットを構成するために，**D-最適設計**を使います．





### D最適設計
* 特徴集合全体をスペクトル(固有値の集合？)的な意味でいい感じにカバーできるサンプリング分布を以下のように定義します．

**定理 3.2** $\mathcal{X} \subset \mathbb{R}^d$ がコンパクト集合（かつフル次元）であると仮定する。このとき、次の特性を持つ $\mathcal{X}$ 上の分布 $\rho$ が存在し，**D-最適設計**という．



* 特性①： $\rho$ は、最大 $d(d+1)/2$ 個の点（すべて $\mathcal{X}$ 内にある）でサポートされています．
  * サポートとは，その点での確率が０より大きい値を取る集合のことです．
  * コンパクト集合とは，**TODO**

* 特性②：$\Sigma = \mathbb{E}_{x \sim \rho} [xx^{\top}]$とした時，任意の$x \in \mathcal{X}$ に対して次が成り立ちます．

$$\|x\|_{\Sigma^{-1}}^2 \leq d$$

  * 直感的に見ると，$x \in \mathcal{X}$のノルムに上界が与えられているので，ベクトルの大きさが一定の大きさ以下であるということですね．


#### D最適設計によるデータセット構築方法

* 各タイムステップ$h$でD最適設計を満たす$s$、$a$の各要素に対して次状態を $P_h(\cdot \mid s, a)$ からサンプリングします。それらと報酬を合わせて$(s, a, r(s, a), s^{\prime})$ とし、データセットを作ります。

* データセットから得られるempricalな共分散行列はフルランクになります。さらにそれは真の共分散行列に対して、半正定値的な意味で大きくなります。解釈としてはデータセットがいい感じに大きな分散を持っていることで学習がうまく行くという感じです。つまり、

$\Lambda_h = \sum_{s, a \in \mathcal{D}_h} \phi(s, a) \phi(s, a)^{\top}$ （empricalな共分散行列）とすると、次が成りたちます。

$$
\Lambda_h = \sum_{s, a \in \mathcal{D}} \phi(s, a) \phi(s, a)^{\top} \succcurlyeq N \Sigma
$$
ということです。

* D-最適設計によって、$\max_{s, a} \phi(s, a)^T \Lambda_h^{-1} \phi(s, a) \leq d$ が成り立ちます。これは大域的最適解を持つことを保証しています。解釈としては、任意の特徴ベクトル $\phi(s, a)$ が、経験的共分散行列の逆行列 $\Lambda_h^{-1}$ によって過度に縮小されないことを意味します。つまり、データセットが偏っていないということが言えます。




### LSVIのサンプル複雑度

**定理 3.3** 特徴がLBCを満たすと仮定します。また、$\delta \in (0, 1)$ および $\epsilon \in (0, 1)$ 、さらにパラメータ $N := \left\lceil\frac{64 H^6 d^2 \ln (1 / \delta)}{\epsilon^2}\right\rceil$とします。 

このとき、少なくとも $1-\delta$の確率で、LSVIは次の性質を持つ $\widehat{\pi}$ を出力します。

$$
\mathbb{E}_{s \sim \mu} V_0^{\star}(s) - \mathbb{E}_{s \sim \mu} V_0^{\widehat{\pi}}(s) \leq \epsilon,
$$

サンプルの総数は $H\left(d^2+\frac{64 H^6 d^2 \ln (1 / \delta)}{\epsilon^2}\right)$ である。

#### 証明(TODO)

### オフラインRLでのLSVIの活用

* データセットから学習をするという意味で、オフラインRLへの応用が考えられます。LSVIをオフラインRLで用いると、次の仮定を置くと、オフライン学習でのサンプル複雑度を導出することができます。

**仮定 3.6 (カバレッジ)** 任意の $h \in [H]$ に対して、

$$
\frac{1}{N} \sum_{(s_i, a_i) \in D_h} \phi(s_i, a_i) \phi(s_i, a_i)^{\top} \succeq \frac{1}{\kappa} \Sigma
$$

とします。この時、

**定理 3.7 (LSVIのサンプル複雑度)** 仮定 3.6 が成り立ち、特徴がLBCを満たすとします。$\delta \in (0, 1)$ および $\epsilon \in (0, 1)$ , $N := \left\lceil\frac{c \kappa H^6 d^2 \ln (1 / \delta)}{\epsilon^2}\right\rceil$ とsします。

ここで、$c$ は定数です。少なくとも $1-\delta$の確率で、

$$
\mathbb{E}_{s \sim \mu} [V_0^{\star}(s)] - \mathbb{E}_{s \sim \mu} [V_0^{\widehat{\pi}}(s)] \leq \epsilon.
$$

をみたす$\widehat{\pi}$ をLSVIは出力します。


**TODO：オフライン方策評価**